In [6]:
!conda install -c conda-forge jpype1
!pip install --upgrade pip
!pip install konlpy 

/bin/bash: conda: command not found


In [7]:
from konlpy.tag import Twitter

In [8]:
twitter = Twitter()
text = "안녕하세요 저의 이름은 최고최건이라고 합니다."
tokens_ko = twitter.morphs(text)

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [9]:
tokens_ko

['안녕하세요', '저', '의', '이름', '은', '최고', '최건', '이라고', '합니다', '.']

In [12]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/ml-data/naver-movie-reviews/ratings_train.txt', sep="\t")
train_df.head(4)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0


In [13]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [15]:
train_df = train_df.dropna(how='any')

In [16]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [17]:
import re

train_df['document'] = train_df['document'].apply(lambda x : re.sub(r"\d+", " ", x))

test_df = pd.read_csv("/content/drive/MyDrive/ml-data/naver-movie-reviews/ratings_test.txt", sep="\t")
test_df = test_df.dropna(how="any")
test_df['document'] = test_df['document'].apply(lambda x : re.sub(r"\d+", " ", x))

train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

In [18]:
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(text):
  tokens_ko = twitter.morphs(text)
  return tokens_ko

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


KeyboardInterrupt: ignored

In [ ]:
lg_clf = LogisticRegression(random_state=0, n_jobs=-1)
params = {'C':[1, 3.5, 4.5, 5.5, 10]}

In [ ]:
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv=3, scoring="accuracy", verbose=1, n_jobs=-1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_, 4))

In [ ]:
from sklearn.metrics import accuracy_score

tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print(accuracy_score(test_df['label'], preds))